# Exercise 6: Building Your First Agent
## Creating a Financial Analyst Agent with Tool Selection

In this notebook, we'll build a sophisticated financial analyst agent that can use multiple calculation tools and make intelligent decisions about which tools to use. We'll explore `create_tool_calling_agent` and `AgentExecutor` for advanced agent functionality.

## Learning Objectives
- Master the `create_tool_calling_agent` function
- Understand `AgentExecutor` for agent orchestration
- Implement intelligent tool selection logic
- Build a comprehensive financial analysis agent
- Handle complex multi-step financial calculations
- Create an interactive financial advisor system

Let's build your first intelligent financial agent! 🤖📊

## Setup and Imports

First, let's import all the necessary libraries for building our financial analyst agent.

In [1]:
# Import necessary libraries
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool, tool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv
import json
import math
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Initialize ChatOpenAI
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,  # Lower temperature for more consistent calculations
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✓ Setup complete!")

✓ Setup complete!


## Building Financial Calculation Tools

Let's create a comprehensive set of financial calculation tools that our agent can use. Each tool will be designed with clear descriptions to help the agent select the right tool for each task.

In [2]:
# Define input models for structured tools
class CompoundInterestInput(BaseModel):
    principal: float = Field(description="Initial investment amount")
    rate: float = Field(description="Annual interest rate (as decimal, e.g., 0.08 for 8%)")
    time: int = Field(description="Number of years")
    compounds_per_year: int = Field(default=12, description="Number of times interest is compounded per year")

class LoanPaymentInput(BaseModel):
    principal: float = Field(description="Loan amount")
    rate: float = Field(description="Annual interest rate (as decimal, e.g., 0.05 for 5%)")
    years: int = Field(description="Loan term in years")

class RetirementInput(BaseModel):
    current_age: int = Field(description="Current age")
    retirement_age: int = Field(description="Planned retirement age")
    current_savings: float = Field(description="Current retirement savings")
    monthly_contribution: float = Field(description="Monthly contribution to retirement")
    annual_return: float = Field(description="Expected annual return (as decimal, e.g., 0.07 for 7%)")
    retirement_goal: float = Field(description="Target retirement savings amount")

class StockAnalysisInput(BaseModel):
    symbol: str = Field(description="Stock ticker symbol (e.g., AAPL, MSFT)")
    period: str = Field(default="1y", description="Time period for analysis (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)")

class PortfolioInput(BaseModel):
    stocks: List[str] = Field(description="List of stock symbols")
    weights: List[float] = Field(description="List of portfolio weights (should sum to 1.0)")
    investment_amount: float = Field(description="Total investment amount")

print("✓ Input models defined!")

✓ Input models defined!


In [3]:
# Create financial calculation tools

@tool
def compound_interest_calculator(principal: float, rate: float, time: int, compounds_per_year: int = 12) -> str:
    """
    Calculate compound interest for investments.
    
    Args:
        principal: Initial investment amount
        rate: Annual interest rate (as decimal, e.g., 0.08 for 8%)
        time: Number of years
        compounds_per_year: Number of times interest is compounded per year (default: 12)
    
    Returns:
        Detailed compound interest calculation results
    """
    try:
        # A = P(1 + r/n)^(nt)
        amount = principal * (1 + rate/compounds_per_year) ** (compounds_per_year * time)
        interest_earned = amount - principal
        
        result = {
            "initial_investment": principal,
            "annual_rate": f"{rate*100:.2f}%",
            "time_period": f"{time} years",
            "compounds_per_year": compounds_per_year,
            "final_amount": round(amount, 2),
            "interest_earned": round(interest_earned, 2),
            "total_return": f"{(interest_earned/principal)*100:.2f}%"
        }
        
        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error calculating compound interest: {str(e)}"

@tool
def loan_payment_calculator(principal: float, rate: float, years: int) -> str:
    """
    Calculate monthly loan payments using the standard loan payment formula.
    
    Args:
        principal: Loan amount
        rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
        years: Loan term in years
    
    Returns:
        Detailed loan payment information
    """
    try:
        monthly_rate = rate / 12
        num_payments = years * 12
        
        if rate == 0:
            monthly_payment = principal / num_payments
        else:
            monthly_payment = principal * (monthly_rate * (1 + monthly_rate)**num_payments) / ((1 + monthly_rate)**num_payments - 1)
        
        total_paid = monthly_payment * num_payments
        total_interest = total_paid - principal
        
        result = {
            "loan_amount": principal,
            "annual_rate": f"{rate*100:.2f}%",
            "loan_term": f"{years} years",
            "monthly_payment": round(monthly_payment, 2),
            "total_paid": round(total_paid, 2),
            "total_interest": round(total_interest, 2),
            "interest_percentage": f"{(total_interest/principal)*100:.2f}%"
        }
        
        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error calculating loan payment: {str(e)}"

@tool
def retirement_calculator(current_age: int, retirement_age: int, current_savings: float, 
                         monthly_contribution: float, annual_return: float, retirement_goal: float) -> str:
    """
    Calculate retirement savings projections and determine if goals will be met.
    
    Args:
        current_age: Current age
        retirement_age: Planned retirement age
        current_savings: Current retirement savings
        monthly_contribution: Monthly contribution to retirement
        annual_return: Expected annual return (as decimal, e.g., 0.07 for 7%)
        retirement_goal: Target retirement savings amount
    
    Returns:
        Detailed retirement planning analysis
    """
    try:
        years_to_retirement = retirement_age - current_age
        monthly_return = annual_return / 12
        total_months = years_to_retirement * 12
        
        # Future value of current savings
        future_current_savings = current_savings * (1 + annual_return) ** years_to_retirement
        
        # Future value of monthly contributions (annuity)
        if monthly_return == 0:
            future_contributions = monthly_contribution * total_months
        else:
            future_contributions = monthly_contribution * (((1 + monthly_return) ** total_months - 1) / monthly_return)
        
        total_retirement_savings = future_current_savings + future_contributions
        
        # Check if goal is met
        goal_met = total_retirement_savings >= retirement_goal
        shortfall = max(0, retirement_goal - total_retirement_savings)
        
        result = {
            "current_age": current_age,
            "retirement_age": retirement_age,
            "years_to_retirement": years_to_retirement,
            "current_savings": current_savings,
            "monthly_contribution": monthly_contribution,
            "annual_return": f"{annual_return*100:.2f}%",
            "projected_retirement_savings": round(total_retirement_savings, 2),
            "retirement_goal": retirement_goal,
            "goal_met": goal_met,
            "shortfall": round(shortfall, 2) if shortfall > 0 else 0,
            "excess": round(total_retirement_savings - retirement_goal, 2) if goal_met else 0
        }
        
        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error calculating retirement projections: {str(e)}"

print("✓ Financial calculation tools created!")

✓ Financial calculation tools created!


In [4]:
# Create stock analysis tools

@tool
def stock_analysis_tool(symbol: str, period: str = "1y") -> str:
    """
    Analyze stock performance and calculate key metrics.
    
    Args:
        symbol: Stock ticker symbol (e.g., AAPL, MSFT)
        period: Time period for analysis (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)
    
    Returns:
        Detailed stock analysis including price metrics and performance
    """
    try:
        # Get stock data
        stock = yf.Ticker(symbol.upper())
        hist = stock.history(period=period)
        info = stock.info
        
        if hist.empty:
            return f"No data available for symbol: {symbol}"
        
        # Calculate metrics
        current_price = hist['Close'].iloc[-1]
        start_price = hist['Close'].iloc[0]
        high_price = hist['High'].max()
        low_price = hist['Low'].min()
        volume_avg = hist['Volume'].mean()
        
        # Performance calculations
        total_return = (current_price - start_price) / start_price
        volatility = hist['Close'].pct_change().std() * np.sqrt(252)  # Annualized volatility
        
        result = {
            "symbol": symbol.upper(),
            "company_name": info.get('longName', 'N/A'),
            "period_analyzed": period,
            "current_price": round(current_price, 2),
            "period_start_price": round(start_price, 2),
            "period_high": round(high_price, 2),
            "period_low": round(low_price, 2),
            "total_return": f"{total_return*100:.2f}%",
            "annualized_volatility": f"{volatility*100:.2f}%",
            "average_volume": int(volume_avg),
            "market_cap": info.get('marketCap', 'N/A'),
            "pe_ratio": info.get('trailingPE', 'N/A'),
            "dividend_yield": info.get('dividendYield', 'N/A')
        }
        
        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error analyzing stock {symbol}: {str(e)}"

@tool
def portfolio_analysis_tool(stocks: List[str], weights: List[float], investment_amount: float) -> str:
    """
    Analyze a portfolio of stocks with given weights.
    
    Args:
        stocks: List of stock symbols
        weights: List of portfolio weights (should sum to 1.0)
        investment_amount: Total investment amount
    
    Returns:
        Portfolio analysis including allocation and performance metrics
    """
    try:
        if len(stocks) != len(weights):
            return "Error: Number of stocks must match number of weights"
        
        if abs(sum(weights) - 1.0) > 0.01:
            return "Error: Weights must sum to 1.0"
        
        portfolio_data = []
        total_value = 0
        
        for i, symbol in enumerate(stocks):
            try:
                stock = yf.Ticker(symbol.upper())
                hist = stock.history(period="1y")
                info = stock.info
                
                if not hist.empty:
                    current_price = hist['Close'].iloc[-1]
                    allocation = investment_amount * weights[i]
                    shares = allocation / current_price
                    
                    # Calculate 1-year return
                    if len(hist) > 1:
                        year_return = (hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]
                    else:
                        year_return = 0
                    
                    portfolio_data.append({
                        "symbol": symbol.upper(),
                        "company_name": info.get('longName', 'N/A'),
                        "weight": f"{weights[i]*100:.1f}%",
                        "allocation": round(allocation, 2),
                        "current_price": round(current_price, 2),
                        "shares": round(shares, 4),
                        "one_year_return": f"{year_return*100:.2f}%"
                    })
                    
                    total_value += allocation
                    
            except Exception as e:
                portfolio_data.append({
                    "symbol": symbol.upper(),
                    "error": f"Could not fetch data: {str(e)}"
                })
        
        result = {
            "total_investment": investment_amount,
            "portfolio_composition": portfolio_data,
            "diversification_score": f"{1 - sum([w**2 for w in weights]):.3f}",  # Herfindahl index
            "number_of_holdings": len(stocks)
        }
        
        return json.dumps(result, indent=2)
    except Exception as e:
        return f"Error analyzing portfolio: {str(e)}"

print("✓ Stock analysis tools created!")

✓ Stock analysis tools created!


## Creating the Financial Analyst Agent

Now let's create our financial analyst agent using `create_tool_calling_agent` and `AgentExecutor`. The agent will be able to intelligently select and use the appropriate tools for different financial analysis tasks.

In [5]:
# Create the list of tools for our agent
tools = [
    compound_interest_calculator,
    loan_payment_calculator,
    retirement_calculator,
    stock_analysis_tool,
    portfolio_analysis_tool
]

# Create a prompt template for our financial analyst agent
prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a professional financial analyst assistant with expertise in:
- Investment calculations and compound interest
- Loan and mortgage analysis
- Retirement planning and projections
- Stock market analysis and portfolio management
- Financial planning and risk assessment

Your role is to:
1. Analyze financial questions and determine the best approach
2. Select and use appropriate calculation tools
3. Provide clear, actionable financial insights
4. Explain complex financial concepts in simple terms
5. Offer practical recommendations based on calculations

When using tools:
- Always verify input parameters make sense
- Use appropriate time periods for different analyses
- Consider risk factors and market conditions
- Provide context for your calculations
- Suggest follow-up questions or additional analyses

Be professional, accurate, and helpful in all your responses.
    """),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create the tool-calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("✓ Financial Analyst Agent created successfully!")
print(f"✓ Agent has access to {len(tools)} financial tools")

✓ Financial Analyst Agent created successfully!
✓ Agent has access to 5 financial tools


## Testing the Financial Analyst Agent

Let's test our agent with various financial scenarios to see how it intelligently selects and uses different tools.

In [6]:
# Test 1: Investment Growth Analysis
print("🔍 TEST 1: Investment Growth Analysis")
print("=" * 50)

query1 = """
I'm 25 years old and want to invest $10,000 in a retirement account. 
If I can get an 8% annual return compounded monthly, 
how much will this grow to by the time I'm 65?
"""

try:
    result1 = agent_executor.invoke({"input": query1})
    print("\n💡 Agent Response:")
    print(result1['output'])
except Exception as e:
    print(f"❌ Error: {e}")

🔍 TEST 1: Investment Growth Analysis


> Entering new AgentExecutor chain...

Invoking: `compound_interest_calculator` with `{'principal': 10000, 'rate': 0.08, 'time': 40, 'compounds_per_year': 12}`


{
  "initial_investment": 10000.0,
  "annual_rate": "8.00%",
  "time_period": "40 years",
  "compounds_per_year": 12,
  "final_amount": 242733.86,
  "interest_earned": 232733.86,
  "total_return": "2327.34%"
}If you invest $10,000 in a retirement account at the age of 25 with an 8% annual return compounded monthly, by the time you're 65, it will grow to approximately $242,733.86. This means you would have earned $232,733.86 in interest, representing a total return of 2327.34%. It's a great way to start building your retirement savings early.

> Finished chain.

💡 Agent Response:
If you invest $10,000 in a retirement account at the age of 25 with an 8% annual return compounded monthly, by the time you're 65, it will grow to approximately $242,733.86. This means you would have earned $232,7

In [7]:
# Test 2: Loan Analysis
print("\n🔍 TEST 2: Mortgage Payment Analysis")
print("=" * 50)

query2 = """
I'm looking at a $300,000 mortgage with a 30-year term at 6.5% interest rate. 
What would my monthly payment be, and how much total interest will I pay?
"""

try:
    result2 = agent_executor.invoke({"input": query2})
    print("\n💡 Agent Response:")
    print(result2['output'])
except Exception as e:
    print(f"❌ Error: {e}")


🔍 TEST 2: Mortgage Payment Analysis


> Entering new AgentExecutor chain...

Invoking: `loan_payment_calculator` with `{'principal': 300000, 'rate': 0.065, 'years': 30}`


{
  "loan_amount": 300000.0,
  "annual_rate": "6.50%",
  "loan_term": "30 years",
  "monthly_payment": 1896.2,
  "total_paid": 682633.47,
  "total_interest": 382633.47,
  "interest_percentage": "127.54%"
}For a $300,000 mortgage with a 30-year term at a 6.5% interest rate:
- Your monthly payment would be approximately $1,896.20.
- The total amount you will pay over the life of the loan is $682,633.47.
- The total interest paid over the 30 years would be $382,633.47, which is about 127.54% of the loan amount.

> Finished chain.

💡 Agent Response:
For a $300,000 mortgage with a 30-year term at a 6.5% interest rate:
- Your monthly payment would be approximately $1,896.20.
- The total amount you will pay over the life of the loan is $682,633.47.
- The total interest paid over the 30 years would be $382,633.47, which is 

In [8]:
# Test 3: Retirement Planning
print("\n🔍 TEST 3: Comprehensive Retirement Planning")
print("=" * 50)

query3 = """
I'm 30 years old with $50,000 already saved for retirement. 
I can contribute $500 per month and expect a 7% annual return. 
I want to retire at 65 with $1.5 million. Will I reach my goal?
"""

try:
    result3 = agent_executor.invoke({"input": query3})
    print("\n💡 Agent Response:")
    print(result3['output'])
except Exception as e:
    print(f"❌ Error: {e}")


🔍 TEST 3: Comprehensive Retirement Planning


> Entering new AgentExecutor chain...

Invoking: `retirement_calculator` with `{'current_age': 30, 'retirement_age': 65, 'current_savings': 50000, 'monthly_contribution': 500, 'annual_return': 0.07, 'retirement_goal': 1500000}`


{
  "current_age": 30,
  "retirement_age": 65,
  "years_to_retirement": 35,
  "current_savings": 50000.0,
  "monthly_contribution": 500.0,
  "annual_return": "7.00%",
  "projected_retirement_savings": 1434356.37,
  "retirement_goal": 1500000.0,
  "goal_met": false,
  "shortfall": 65643.63,
  "excess": 0
}Based on the calculations, here's the analysis of your retirement plan:

- **Current Age:** 30
- **Retirement Age:** 65
- **Years to Retirement:** 35
- **Current Savings:** $50,000
- **Monthly Contribution:** $500
- **Annual Return:** 7.00%

### Retirement Savings Projection:
- **Projected Retirement Savings:** $1,434,356.37
- **Retirement Goal:** $1,500,000.00

### Conclusion:
- **Goal Met:** No
- **Shortfall:** 

In [9]:
# Test 4: Stock Analysis
print("\n🔍 TEST 4: Stock Performance Analysis")
print("=" * 50)

query4 = """
Can you analyze the performance of Apple (AAPL) stock over the past year? 
I'm interested in its price movement, volatility, and key metrics.
"""

try:
    result4 = agent_executor.invoke({"input": query4})
    print("\n💡 Agent Response:")
    print(result4['output'])
except Exception as e:
    print(f"❌ Error: {e}")


🔍 TEST 4: Stock Performance Analysis


> Entering new AgentExecutor chain...

Invoking: `stock_analysis_tool` with `{'symbol': 'AAPL', 'period': '1y'}`


{
  "symbol": "AAPL",
  "company_name": "Apple Inc.",
  "period_analyzed": "1y",
  "current_price": 213.55,
  "period_start_price": 225.29,
  "period_high": 259.47,
  "period_low": 168.99,
  "total_return": "-5.21%",
  "annualized_volatility": "32.14%",
  "average_volume": 53657986,
  "market_cap": 3189540126720,
  "pe_ratio": 33.21151,
  "dividend_yield": 0.51
}### Apple Inc. (AAPL) Stock Analysis (Past Year)

- **Company Name:** Apple Inc.
- **Current Price:** $213.55
- **Period Start Price:** $225.29
- **Period High:** $259.47
- **Period Low:** $168.99
- **Total Return:** -5.21%
- **Annualized Volatility:** 32.14%
- **Average Volume:** 53,657,986
- **Market Cap:** $3.19 trillion
- **P/E Ratio:** 33.21
- **Dividend Yield:** 0.51%

#### Insights:
- **Price Movement:** Apple's stock price started the year at $225.29, reached a high o

In [10]:
# Test 5: Portfolio Analysis
print("\n🔍 TEST 5: Portfolio Diversification Analysis")
print("=" * 50)

query5 = """
I want to create a diversified portfolio with $100,000. 
Please analyze this allocation: 40% AAPL, 30% MSFT, 20% GOOGL, 10% TSLA.
How would this portfolio be structured?
"""

try:
    result5 = agent_executor.invoke({"input": query5})
    print("\n💡 Agent Response:")
    print(result5['output'])
except Exception as e:
    print(f"❌ Error: {e}")


🔍 TEST 5: Portfolio Diversification Analysis


> Entering new AgentExecutor chain...

Invoking: `portfolio_analysis_tool` with `{'stocks': ['AAPL', 'MSFT', 'GOOGL', 'TSLA'], 'weights': [0.4, 0.3, 0.2, 0.1], 'investment_amount': 100000}`


{
  "total_investment": 100000.0,
  "portfolio_composition": [
    {
      "symbol": "AAPL",
      "company_name": "Apple Inc.",
      "weight": "40.0%",
      "allocation": 40000.0,
      "current_price": 213.55,
      "shares": 187.3098,
      "one_year_return": "-5.21%"
    },
    {
      "symbol": "MSFT",
      "company_name": "Microsoft Corporation",
      "weight": "30.0%",
      "allocation": 30000.0,
      "current_price": 498.84,
      "shares": 60.1395,
      "one_year_return": "7.51%"
    },
    {
      "symbol": "GOOGL",
      "company_name": "Alphabet Inc.",
      "weight": "20.0%",
      "allocation": 20000.0,
      "current_price": 179.53,
      "shares": 111.402,
      "one_year_return": "-5.35%"
    },
    {
      "symbol": "TSLA",
 

## Advanced Agent Features

Let's explore more advanced capabilities of our agent, including multi-step reasoning and complex financial planning scenarios.

In [11]:
# Test 6: Multi-step Financial Planning
print("\n🔍 TEST 6: Multi-step Financial Planning")
print("=" * 50)

query6 = """
I'm 28 years old and need help with a comprehensive financial plan:
1. I want to buy a house in 5 years for $400,000 with a 20% down payment
2. I currently have $20,000 saved and can save $1,000/month
3. I can earn 6% annually on my savings
4. After buying the house, I'll have a $320,000 mortgage at 7% for 30 years

Can you help me understand:
- Will I have enough for the down payment?
- What will my monthly mortgage payment be?
- How much will I pay in total for the house?
"""

try:
    result6 = agent_executor.invoke({"input": query6})
    print("\n💡 Agent Response:")
    print(result6['output'])
except Exception as e:
    print(f"❌ Error: {e}")


🔍 TEST 6: Multi-step Financial Planning


> Entering new AgentExecutor chain...

Invoking: `retirement_calculator` with `{'current_age': 28, 'retirement_age': 33, 'current_savings': 20000, 'monthly_contribution': 1000, 'annual_return': 0.06, 'retirement_goal': 400000}`


{
  "current_age": 28,
  "retirement_age": 33,
  "years_to_retirement": 5,
  "current_savings": 20000.0,
  "monthly_contribution": 1000.0,
  "annual_return": "6.00%",
  "projected_retirement_savings": 96534.54,
  "retirement_goal": 400000.0,
  "goal_met": false,
  "shortfall": 303465.46,
  "excess": 0
}
Invoking: `loan_payment_calculator` with `{'principal': 320000, 'rate': 0.07, 'years': 30}`


{
  "loan_amount": 320000.0,
  "annual_rate": "7.00%",
  "loan_term": "30 years",
  "monthly_payment": 2128.97,
  "total_paid": 766428.47,
  "total_interest": 446428.47,
  "interest_percentage": "139.51%"
}Based on the analysis:

1. **Down Payment for the House:**
   - With your current savings of $20,000 and monthly contribut

In [12]:
# Test 7: Investment Comparison
print("\n🔍 TEST 7: Investment Strategy Comparison")
print("=" * 50)

query7 = """
I have $50,000 to invest and I'm comparing two strategies:
Strategy A: Invest everything in a savings account at 4% annual interest
Strategy B: Invest in a diversified portfolio expecting 8% annual return

Both for 20 years. Can you calculate the difference and help me understand 
which strategy would be better for long-term wealth building?
"""

try:
    result7 = agent_executor.invoke({"input": query7})
    print("\n💡 Agent Response:")
    print(result7['output'])
except Exception as e:
    print(f"❌ Error: {e}")


🔍 TEST 7: Investment Strategy Comparison


> Entering new AgentExecutor chain...

Invoking: `compound_interest_calculator` with `{'principal': 50000, 'rate': 0.04, 'time': 20}`


{
  "initial_investment": 50000.0,
  "annual_rate": "4.00%",
  "time_period": "20 years",
  "compounds_per_year": 12,
  "final_amount": 111129.1,
  "interest_earned": 61129.1,
  "total_return": "122.26%"
}
Invoking: `compound_interest_calculator` with `{'principal': 50000, 'rate': 0.08, 'time': 20}`


{
  "initial_investment": 50000.0,
  "annual_rate": "8.00%",
  "time_period": "20 years",
  "compounds_per_year": 12,
  "final_amount": 246340.14,
  "interest_earned": 196340.14,
  "total_return": "392.68%"
}After 20 years:
- Strategy A (4% annual interest) would grow your $50,000 investment to approximately $111,129.10, with an interest earned of $61,129.10 (a total return of 122.26%).
- Strategy B (8% annual return) would grow your $50,000 investment to approximately $246,340.14, with an interest earned of $19

## Interactive Financial Assistant

Let's create an interactive function that allows you to ask the agent any financial question and see how it intelligently selects and uses tools.

In [13]:
def ask_financial_advisor(question: str) -> None:
    """
    Interactive function to ask the financial advisor agent any question.
    """
    print(f"❓ Question: {question}")
    print("=" * 80)
    
    try:
        # Get response from agent
        result = agent_executor.invoke({"input": question})
        
        print("\n🤖 Financial Advisor Response:")
        print("-" * 40)
        print(result['output'])
        
        # Show intermediate steps if available
        if 'intermediate_steps' in result and result['intermediate_steps']:
            print("\n🔧 Tools Used:")
            for i, (action, observation) in enumerate(result['intermediate_steps'], 1):
                print(f"  {i}. {action.tool}: {action.tool_input}")
                
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("\n" + "=" * 80 + "\n")

# Example usage
ask_financial_advisor(
    "If I invest $25,000 today at 9% annual return compounded quarterly, "
    "how much will I have in 15 years?"
)

❓ Question: If I invest $25,000 today at 9% annual return compounded quarterly, how much will I have in 15 years?


> Entering new AgentExecutor chain...

Invoking: `compound_interest_calculator` with `{'principal': 25000, 'rate': 0.09, 'time': 15, 'compounds_per_year': 4}`


{
  "initial_investment": 25000.0,
  "annual_rate": "9.00%",
  "time_period": "15 years",
  "compounds_per_year": 4,
  "final_amount": 95003.37,
  "interest_earned": 70003.37,
  "total_return": "280.01%"
}If you invest $25,000 today at an annual return of 9% compounded quarterly, you will have approximately $95,003.37 in 15 years. This means you would have earned $70,003.37 in interest, resulting in a total return of 280.01%.

> Finished chain.

🤖 Financial Advisor Response:
----------------------------------------
If you invest $25,000 today at an annual return of 9% compounded quarterly, you will have approximately $95,003.37 in 15 years. This means you would have earned $70,003.37 in interest, resulting in a to

In [14]:
# Try your own questions!
ask_financial_advisor(
    "I'm 35 and want to retire at 60 with $2 million. "
    "I have $100,000 saved and can invest $1,200/month. "
    "What return do I need to achieve this goal?"
)

❓ Question: I'm 35 and want to retire at 60 with $2 million. I have $100,000 saved and can invest $1,200/month. What return do I need to achieve this goal?


> Entering new AgentExecutor chain...

Invoking: `retirement_calculator` with `{'current_age': 35, 'retirement_age': 60, 'current_savings': 100000, 'monthly_contribution': 1200, 'annual_return': 0.07, 'retirement_goal': 2000000}`


{
  "current_age": 35,
  "retirement_age": 60,
  "years_to_retirement": 25,
  "current_savings": 100000.0,
  "monthly_contribution": 1200.0,
  "annual_return": "7.00%",
  "projected_retirement_savings": 1514829.3,
  "retirement_goal": 2000000.0,
  "goal_met": false,
  "shortfall": 485170.7,
  "excess": 0
}To retire at 60 with $2 million, considering your current age of 35, $100,000 in savings, and monthly contributions of $1,200, you would need an annual return of approximately 7.00% to reach your goal.

Based on the projections:
- Projected retirement savings at age 60: $1,514,829.30
- Shortfall from y

In [15]:
# Test the agent's tool selection intelligence
ask_financial_advisor(
    "Compare the performance of Tesla (TSLA) vs Microsoft (MSFT) "
    "over the past 2 years. Which has been more volatile?"
)

❓ Question: Compare the performance of Tesla (TSLA) vs Microsoft (MSFT) over the past 2 years. Which has been more volatile?


> Entering new AgentExecutor chain...

Invoking: `stock_analysis_tool` with `{'symbol': 'TSLA', 'period': '2y'}`


{
  "symbol": "TSLA",
  "company_name": "Tesla, Inc.",
  "period_analyzed": "2y",
  "current_price": 315.35,
  "period_start_price": 282.48,
  "period_high": 488.54,
  "period_low": 138.8,
  "total_return": "11.64%",
  "annualized_volatility": "63.64%",
  "average_volume": 104311305,
  "market_cap": 1015729750016,
  "pe_ratio": 182.28323,
  "dividend_yield": "N/A"
}
Invoking: `stock_analysis_tool` with `{'symbol': 'MSFT', 'period': '2y'}`


{
  "symbol": "MSFT",
  "company_name": "Microsoft Corporation",
  "period_analyzed": "2y",
  "current_price": 498.84,
  "period_start_price": 332.97,
  "period_high": 500.76,
  "period_low": 305.36,
  "total_return": "49.81%",
  "annualized_volatility": "22.97%",
  "average_volume": 22397469,
  "market_cap": 37

## Understanding Agent Behavior

Let's examine how the agent makes decisions about tool selection and create a function to analyze its reasoning process.

In [16]:
def analyze_agent_decision_making(question: str) -> None:
    """
    Analyze how the agent makes decisions about tool selection.
    """
    print(f"🔍 Analyzing Agent Decision Making")
    print(f"📋 Question: {question}")
    print("=" * 80)
    
    try:
        # Get response with intermediate steps
        result = agent_executor.invoke({"input": question})
        
        print("\n🧠 Agent Reasoning Process:")
        print("-" * 40)
        
        if 'intermediate_steps' in result and result['intermediate_steps']:
            for i, (action, observation) in enumerate(result['intermediate_steps'], 1):
                print(f"\n📍 Step {i}: Tool Selection")
                print(f"   🔧 Tool: {action.tool}")
                print(f"   📊 Input: {action.tool_input}")
                print(f"   📈 Output: {observation[:200]}..." if len(str(observation)) > 200 else f"   📈 Output: {observation}")
                
        print("\n🎯 Final Response:")
        print("-" * 40)
        print(result['output'])
        
        # Analysis summary
        if 'intermediate_steps' in result and result['intermediate_steps']:
            tools_used = [action.tool for action, _ in result['intermediate_steps']]
            print(f"\n📊 Agent Performance Summary:")
            print(f"   🎯 Tools Used: {', '.join(tools_used)}")
            print(f"   🔢 Number of Steps: {len(result['intermediate_steps'])}")
            print(f"   ✅ Task Completed Successfully")
            
    except Exception as e:
        print(f"❌ Error in analysis: {e}")
    
    print("\n" + "=" * 80 + "\n")

# Test the analysis function
analyze_agent_decision_making(
    "I need to decide between two investment options: "
    "$10,000 in a CD at 5% for 10 years, or "
    "$10,000 in an index fund expecting 7% annually for 10 years. "
    "Which is better?"
)

🔍 Analyzing Agent Decision Making
📋 Question: I need to decide between two investment options: $10,000 in a CD at 5% for 10 years, or $10,000 in an index fund expecting 7% annually for 10 years. Which is better?


> Entering new AgentExecutor chain...

Invoking: `compound_interest_calculator` with `{'principal': 10000, 'rate': 0.05, 'time': 10}`


{
  "initial_investment": 10000.0,
  "annual_rate": "5.00%",
  "time_period": "10 years",
  "compounds_per_year": 12,
  "final_amount": 16470.09,
  "interest_earned": 6470.09,
  "total_return": "64.70%"
}
Invoking: `compound_interest_calculator` with `{'principal': 10000, 'rate': 0.07, 'time': 10}`


{
  "initial_investment": 10000.0,
  "annual_rate": "7.00%",
  "time_period": "10 years",
  "compounds_per_year": 12,
  "final_amount": 20096.61,
  "interest_earned": 10096.61,
  "total_return": "100.97%"
}After 10 years:
- Investing $10,000 in a CD at 5% annually would grow to approximately $16,470.09.
- Investing $10,000 in an index fund at 7% 

## Advanced Agent Customization

Let's create a more specialized version of our agent with enhanced capabilities and custom system prompts.

In [17]:
# Create a specialized retirement planning agent
retirement_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a specialized retirement planning advisor with deep expertise in:
- 401(k) and IRA planning strategies
- Social Security optimization
- Asset allocation for different life stages
- Tax-efficient retirement strategies
- Healthcare cost planning in retirement

Your approach should be:
1. Comprehensive - consider all aspects of retirement planning
2. Personalized - tailor advice to individual circumstances
3. Risk-aware - account for market volatility and inflation
4. Actionable - provide specific, implementable recommendations
5. Educational - explain the reasoning behind recommendations

Always consider factors like:
- Current age and retirement timeline
- Risk tolerance and investment horizon
- Expected retirement lifestyle and expenses
- Healthcare and long-term care needs
- Tax implications of different strategies
- Inflation impact on purchasing power

Provide detailed calculations and multiple scenarios when helpful.
    """),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create specialized retirement agent
retirement_agent = create_tool_calling_agent(llm, tools, retirement_prompt)
retirement_executor = AgentExecutor(
    agent=retirement_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    max_iterations=15,
    handle_parsing_errors=True
)

print("✓ Specialized Retirement Planning Agent created!")

✓ Specialized Retirement Planning Agent created!


In [18]:
# Test the specialized retirement agent
print("🔍 Testing Specialized Retirement Planning Agent")
print("=" * 60)

retirement_query = """
I'm 45 years old, married, and want to retire at 62. Here's my situation:
- Current retirement savings: $250,000
- Monthly contribution capacity: $2,000
- Expected annual return: 6.5%
- Desired retirement income: $80,000/year
- Life expectancy: 85 years

Can you create a comprehensive retirement strategy and tell me:
1. Will I have enough saved by 62?
2. How long will my money last in retirement?
3. What adjustments might I need to make?
"""

try:
    retirement_result = retirement_executor.invoke({"input": retirement_query})
    print("\n🎯 Retirement Planning Analysis:")
    print(retirement_result['output'])
except Exception as e:
    print(f"❌ Error: {e}")

🔍 Testing Specialized Retirement Planning Agent


> Entering new AgentExecutor chain...

Invoking: `retirement_calculator` with `{'current_age': 45, 'retirement_age': 62, 'current_savings': 250000, 'monthly_contribution': 2000, 'annual_return': 0.065, 'retirement_goal': 80000}`


{
  "current_age": 45,
  "retirement_age": 62,
  "years_to_retirement": 17,
  "current_savings": 250000.0,
  "monthly_contribution": 2000.0,
  "annual_return": "6.50%",
  "projected_retirement_savings": 1471502.1,
  "retirement_goal": 80000.0,
  "goal_met": true,
  "shortfall": 0,
  "excess": 1391502.1
}Based on the analysis of your retirement plan:
1. **Retirement Savings by Age 62**:
   - With your current savings of $250,000, monthly contributions of $2,000, and an expected annual return of 6.5%, you are projected to have approximately $1,471,502 saved by the time you reach 62.
   - You are on track to meet your retirement goal of $80000/year with a surplus of $1,391,502 by the time you retire.

2. **Money 

## Summary and Key Learnings

### What We've Built:
1. **Comprehensive Financial Tools**: Created specialized tools for different financial calculations
2. **Intelligent Agent**: Built an agent that can select appropriate tools based on user queries
3. **Advanced Reasoning**: Implemented multi-step financial planning capabilities
4. **Specialized Agents**: Created domain-specific agents for specialized advice

### Key Concepts Learned:
- **`create_tool_calling_agent`**: Creates agents that can intelligently select and use tools
- **`AgentExecutor`**: Orchestrates agent execution with error handling and iteration control
- **Tool Selection Logic**: How agents decide which tools to use based on context
- **Multi-step Reasoning**: Agents can chain multiple tool calls to solve complex problems
- **Prompt Engineering**: Specialized system prompts create domain-specific expertise

### Best Practices:
1. **Clear Tool Descriptions**: Help agents understand when to use each tool
2. **Comprehensive System Prompts**: Define agent expertise and behavior
3. **Error Handling**: Implement robust error handling for tool failures
4. **Iteration Limits**: Set reasonable limits to prevent infinite loops
5. **Intermediate Steps**: Track agent reasoning for debugging and transparency

## Exercises and Experiments

Try these experiments to deepen your understanding:

In [19]:
# Exercise 1: Create your own custom tool
# TODO: Create a new financial tool (e.g., inflation calculator, budget planner, etc.)
# Add it to the tools list and test it with the agent

# Your code here:
pass

In [20]:
# Exercise 2: Test edge cases
# TODO: Test the agent with edge cases like:
# - Very large numbers
# - Zero or negative values
# - Invalid stock symbols
# - Impossible scenarios

# Your code here:
pass

In [21]:
# Exercise 3: Create a specialized agent
# TODO: Create a specialized agent for a specific domain like:
# - Real estate investment analysis
# - Small business financial planning
# - College savings planning

# Your code here:
pass

## Next Steps

1. **Explore More Tools**: Add more sophisticated financial analysis tools
2. **Enhance Prompts**: Create more specialized system prompts for different scenarios
3. **Add Memory**: Implement conversation memory for contextual follow-up questions
4. **Create Workflows**: Build multi-agent workflows for complex financial planning
5. **Add Visualization**: Integrate charts and graphs for better data presentation

Congratulations! You've built a sophisticated financial analyst agent that can intelligently select and use tools to provide comprehensive financial advice! 🎉💰